In [1]:
import datetime as dt
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import APSIMGraphHelpers as AGH
import GraphHelpers as GH
from scipy import stats
import statsmodels.api as sm
from statsmodels.formula.api import ols
import matplotlib.dates as mdates
import MathsUtilities as MUte
import shlex # package to construct the git command to subprocess format
import subprocess 
#import ProcessWheatFiles as pwf
import xmltodict, json
import sqlite3
import scipy.optimize 
from skopt import gp_minimize
from skopt.callbacks import CheckpointSaver
from skopt import load
from skopt.plots import plot_convergence
import matplotlib.gridspec as gridspec

import winsound
frequency = 2500  # Set Frequency To 2500 Hertz
duration = 1000  # Set Duration To 1000 ms == 1 second
%matplotlib inline

In [2]:
WheatFilePath = r"C:/GitHubRepos/ApsimX/Tests/Validation/Wheat/Wheat.apsimx"
DBFilePath = r"C:/GitHubRepos/ApsimX/Tests/Validation/Wheat/Wheat.db"

BaseLine = pd.read_pickle('./BaselineHarvestObsPred.pkl')

DailyBaseLine = pd.read_pickle('./BaselineDailyObsPred.pkl')

BZOptimised = pd.read_pickle('./CAMP_optimisedBZHarvestObsPred.pkl')

paramNames = ['[Phenology].CAMP.FLNparams.MinLN', 
              '[Phenology].CAMP.FLNparams.PpLN', 
              '[Phenology].CAMP.FLNparams.VrnLN', 
              '[Phenology].CAMP.FLNparams.VxPLN',
              '[Phenology].HeadEmergenceLongDayBase.FixedValue',
              '[Phenology].HeadEmergencePpSensitivity.FixedValue']

FittingVariables = ['Wheat.Phenology.FinalLeafNumber','Wheat.Phenology.FlagLeafDAS',
                    'Wheat.Phenology.HeadingDAS','Wheat.Phenology.FloweringDAS']        

BlankManager = {'$type': 'Models.Manager, Models',
            'Code': '',
            'Parameters': None,
            'Name': 'SetCropParameters',
            'IncludeInDocumentation': False,
            'Enabled': True,
            'ReadOnly': False}

SetCropParams = {
          "$type": "Models.Manager, Models",
          "Code": "using Models.Core;\r\nusing System;\r\nnamespace Models\r\n{\r\n\t[Serializable]\r\n    public class Script : Model\r\n    {\r\n        [Link] Zone zone;\r\n        [EventSubscribe(\"PlantSowing\")]\r\n        private void OnPlantSowing(object sender, EventArgs e)\r\n        {\r\n            object PpFac12 = 0.8;\r\n            zone.Set(\"Wheat.Phenology.CAMP.PpResponse.XYPairs.Y[3]\", PpFac12);  \r\n            object DeVernFac = -.3;\r\n            zone.Set(\"Wheat.Phenology.CAMP.DailyColdVrn1.Response.DeVernalisationRate.FixedValue\", DeVernFac);  \r\n        }\r\n    }\r\n}\r\n                \r\n",
          "Parameters": [],
          "Name": "SetCropParameters",
          "IncludeInDocumentation": False,
          "Enabled": True,
          "ReadOnly": False}

def AppendModeltoModelofTypeAndDeleteOldIfPresent(Parent,TypeToAppendTo,ModelToAppend):
    try:
        for child in Parent['Children']:
            if child['$type'] == TypeToAppendTo:
                pos = 0
                for g in child['Children']:
                    if g['Name'] == ModelToAppend['Name']:
                        del child['Children'][pos]
                        #print('Model ' + ModelToAppend['Name'] + ' found and deleted')
                    pos+=1
                child['Children'].append(ModelToAppend)
                return True
            else:
                Parent = AppendModeltoModelofTypeAndDeleteOldIfPresent(child,TypeToAppendTo,ModelToAppend)
        return Parent
    except:
        return Parent
    
def AppendModeltoModelofType(Parent,TypeToAppendTo,NameToAppendTo,ModelToAppend):
    try:
        for child in Parent['Children']:
            if (child['$type'] == TypeToAppendTo) and (child['Name'] == NameToAppendTo):
                child['Children'].append(ModelToAppend)
                return True
            else:
                Parent = AppendModeltoModelofType(child,TypeToAppendTo,NameToAppendTo,ModelToAppend)
        return Parent
    except:
        return Parent
    
def findNextChild(Parent,ChildName):
    if len(Parent['Children']) >0:
        for child in range(len(Parent['Children'])):
            if Parent['Children'][child]['Name'] == ChildName:
                return Parent['Children'][child]
    else:
        return Parent[ChildName]

def findModel(Parent,PathElements):
    for pe in PathElements:
        Parent = findNextChild(Parent,pe)
    return Parent    

def StopReporting(WheatApsimx,modelPath):
    PathElements = modelPath.split('.')
    report = findModel(WheatApsimx,PathElements)
    report["EventNames"] = []

def removeModel(Parent,modelPath):
    PathElements = modelPath.split('.')
    Parent = findModel(Parent,PathElements[:-1])
    pos = 0
    found = False
    for c in Parent['Children']:
        if c['Name'] == PathElements[-1]:
            del Parent['Children'][pos]
            found = True
            break
        pos += 1
    if found == False:
        print('Failed to find ' + PathElements[-1] + ' to delete')

def ApplyParamReplacementSet(paramValues,paramNames,filePath):
    with open(filePath,'r') as WheatApsimxJSON:
        WheatApsimx = json.load(WheatApsimxJSON)
        WheatApsimxJSON.close()
    ## Remove old prameterSet manager in replacements
    removeModel(WheatApsimx,'Replacements.SetCropParams')

    ## Add crop coefficient overwrite into replacements
    codeString = "using Models.Core;\r\nusing System;\r\nnamespace Models\r\n{\r\n\t[Serializable]\r\n    public class Script : Model\r\n    {\r\n        [Link] Zone zone;\r\n        [EventSubscribe(\"Sowing\")]\r\n        private void OnSowing(object sender, EventArgs e)\r\n     {\r\n        object Pval = 0; \r\n    "
    for p in range(len(paramValues)):
        codeString +=  "         Pval ="
        codeString += str(paramValues[p])
        codeString += ';\r\n         zone.Set(\"'
        codeString += paramNames[p]
        codeString += '\", Pval);  \r\n'
        
    codeString += '\r\n}\r\n}\r\n  }'

    SetCropParams["Code"] = codeString

    AppendModeltoModelofType(WheatApsimx,"Models.Core.Folder, Models","Replacements",SetCropParams)

    with open(filePath,'w') as WheatApsimxJSON:
        json.dump(WheatApsimx,WheatApsimxJSON,indent=2)
        
def makeLongString(SimulationSet):
    longString =  '/SimulationNameRegexPattern:"'
    longString =  longString + '(' + SimulationSet[0]  + ')|' # Add first on on twice as apsim doesn't run the first in the list
    for sim in SimulationSet[:]:
        longString = longString + '(' + sim + ')|'
    longString = longString + '(' + SimulationSet[-1] + ')|' ## Add Last on on twice as apsim doesnt run the last in the list
    longString = longString + '(' + SimulationSet[-1] + ')"'
    return longString

def CalcScaledValue(Value,RMax,RMin):
    return (Value - RMin)/(RMax-RMin)

In [3]:
def Preparefile(filePath,freq):
    ## revert .apximx file to master
    !del C:\GitHubRepos\ApsimX\Tests\Validation\Wheat\Wheat.db
    #command= "git --git-dir=C:/GitHubRepos/ApsimX/.git --work-tree=C:/GitHubRepos/ApsimX checkout " + filePath 
    #comm=shlex.split(command) # This will convert the command into list format
    #subprocess.run(comm, shell=True) 
    ## Add blank manager into each simulation
    with open(filePath,'r') as WheatApsimxJSON:
        WheatApsimx = json.load(WheatApsimxJSON)
    WheatApsimxJSON.close()
    if freq == 'Harvest':
        #Stop Daily reporting
        StopReporting(WheatApsimx,'Replacements.DailyReport')
        removeModel(WheatApsimx,'DataStore.DailyObsPred')
    else:
        if freq != 'Daily':
            print('Only works with Daily or Harvest frequencies')
    AppendModeltoModelofTypeAndDeleteOldIfPresent(WheatApsimx,'Models.Core.Zone, Models',BlankManager)
    with open(filePath,'w') as WheatApsimxJSON:
        json.dump(WheatApsimx,WheatApsimxJSON,indent=2)
    print('File Prep Complete')
    
def runModelItter(paramNames,paramValues,FittingVariables,Cultivar,DataTable,freq,WheatFilePath,ver):
    paramNames = paramNames + ['[Phenology].CAMP.ColdVrnResponse.Response.DeVernalisationRate.FixedValue']
    paramValues = paramValues + [-5.0]
    ApplyParamReplacementSet(paramValues,paramNames,WheatFilePath)
    OptimisationVariables = ['Observed.'+x for x in FittingVariables]
    DataPresent = pd.Series(index = DataTable.index,dtype=bool)
    DataPresent = False
    for v in OptimisationVariables:
        DataPresent = (DataPresent | ~np.isnan(pd.to_numeric(DataTable.loc[:,v])))
    SetFilter = (DataTable.Cultivar==Cultivar) & DataPresent
    SimulationSet = DataTable.loc[SetFilter,'SimulationName'].values
    SimSet = makeLongString(SimulationSet)
    #print(SimSet)
    start = dt.datetime.now()
    subprocess.run(['C:/GitHubRepos/ApsimX/bin/Debug/netcoreapp3.1/Models.exe',
                    WheatFilePath,
                    SimSet], stdout=subprocess.PIPE, stderr=subprocess.STDOUT)
    endrun = dt.datetime.now()
    runtime = (endrun-start).seconds
    con = sqlite3.connect(DBFilePath)
    if freq == 'Harvest':
        ObsPred = pd.read_sql("Select * from HarvestObsPred",con)
    else:
        if freq == 'Daily':
            ObsPred = pd.read_sql("Select * from DailyObsPred",con)
        else:
            print('Only works with Daily or Harvest DataTables')
    con.close()
    n = len(ObsPred.loc[:,'Predicted.Simulation.Name'].drop_duplicates())        
    
    ScObsPre = pd.DataFrame(columns = ['ScObs','ScPred','Var','Predicted.Simulation.Name','Predicted.Experiment','Predicted.Wheat.SowingDate'])
    indloc = 0
    for var in FittingVariables:
        DataPairs = ObsPred.reindex(['Observed.'+var,'Predicted.'+var,'Predicted.Simulation.Name','Predicted.Experiment','Predicted.Wheat.SowingDate'],axis=1).dropna()
        for v in ['Observed.'+var,'Predicted.'+var]:
            DataPairs.loc[:,v] = pd.to_numeric(DataPairs.loc[:,v])
        VarMax = max(DataPairs.loc[:,'Observed.'+var].max(),DataPairs.loc[:,'Predicted.'+var].max())
        VarMin = min(DataPairs.loc[:,'Observed.'+var].min(),DataPairs.loc[:,'Predicted.'+var].min())
        for x in DataPairs.index:
            ScObsPre.loc[indloc,'ScObs'] = CalcScaledValue(DataPairs.loc[x,'Observed.'+var],VarMax,VarMin)
            ScObsPre.loc[indloc,'ScPred'] = CalcScaledValue(DataPairs.loc[x,'Predicted.'+var],VarMax,VarMin)
            ScObsPre.loc[indloc,'Var'] = var
            globals()['IttersObsPred'].loc[(Cultivar,globals()['itter'],indloc),['ScObs','ScPred','Var']] = ScObsPre.loc[indloc,['ScObs','ScPred','Var']]
            globals()['IttersObsPred'].loc[(Cultivar,globals()['itter'],indloc),
                                          ['Predicted.Simulation.Name','Predicted.Experiment','Predicted.Wheat.SowingDate']] = DataPairs.loc[x,['Predicted.Simulation.Name','Predicted.Experiment','Predicted.Wheat.SowingDate']]
            indloc+=1
    RegStats = MUte.MathUtilities.CalcRegressionStats('LN',ScObsPre.loc[:,'ScPred'].values,ScObsPre.loc[:,'ScObs'].values)
    globals()['IttersObsPred'].loc[(Cultivar,globals()['itter']),['NSE','nSims']] = [RegStats.NSE,n]
    globals()['IttersObsPred'].loc[(Cultivar,globals()['itter']),paramNames] = paramValues
    try:
        retVal = max(RegStats.NSE,-2) *-1
        print(str(globals()['itter']) +"  "+ str(paramValues) + " run completed " +str(n) + ' sims in '+ str(runtime) + ' seconds.  NSE = '+str(RegStats.NSE))
    except:
        retVal = 2
        print(str(globals()['itter']) +"  "+ str(paramValues) + " run completed " +str(n) + ' sims in '+ str(runtime) + ' seconds.  NSE = insufficient data')
    globals()['IttersObsPred'].to_pickle("./OptFiles/"+c+"_IttersObsPred"+ver+".pkl")
    globals()['itter'] +=1
    return retVal

def runDevModelItter(paramValues):
    LV = paramValues[0]
    SV = paramValues[0] + paramValues[1]
    SN = paramValues[0] + paramValues[1] + paramValues[2]
    LN = paramValues[0] + paramValues[2] + paramValues[3]
    boundsPass = True
    if (LN > 30):
        boundsPass = False
    if (SV > 25):
        boundsPass = False
    if (SN > 35):
        boundsPass = False
    if (SN < LV):
        boundsPass = False
    if (LN < LV):
        boundsPass = False
    if boundsPass == False:
        print (str(itter) +"  "+ str(paramValues) + " gave out of bounds parameters ")
        retVal = 2.111
        globals()['itter'] +=1
    else:
        retVal = runModelItter(paramNames,paramValues,FittingVariables,c,BaseLine,'Harvest',WheatFilePath,Version)
    return retVal

In [12]:
LaTrobeCampInputs = pd.read_excel('C:\\GitHubRepos\\npi\\Analysis\\Controlled Environment\\CEParamsFLN_Aus.xlsx',sheet_name='ObservedParams',index_col='Cultivar',engine='openpyxl',usecols="A:H")
#LaTrobeCampInputs.columns = ['FLN_LV', 'FLN_LN', 'FLN_SV', 'FLN_SN','VrnTreatTemp','VrnTreatDuration','Expt']
LincolnCampInputs = pd.read_excel('C:\\GitHubRepos\\npi\\Analysis\\Controlled Environment\\CEParamsFLN_NZ96.xlsx',sheet_name='ObservedParams',index_col='Cultivar',engine='openpyxl',usecols="A:H")
#LincolnCampInputs.columns = ['FLN_LV', 'FLN_LN', 'FLN_SV', 'FLN_SN','VrnTreatTemp','VrnTreatDuration','Expt']
CampInputs = pd.concat([LaTrobeCampInputs,LincolnCampInputs])
Aus_FL_ANdata = pd.read_excel('C:\\GitHubRepos\\npi\\Simulation\\ModelFitting\\FinalNPIFitting.xlsx',sheet_name='FL_AN',index_col='Row Labels',engine='openpyxl',usecols="A:C",skiprows=3)
for c in CampInputs.index:
    try:
        CampInputs.loc[c,'[Phenology].HeadEmergenceLongDayBase.FixedValue'] = Aus_FL_ANdata.loc[c,'[Phenology].HeadEmergenceLongDayBase.FixedValue']
    except:
        CampInputs.loc[c,'[Phenology].HeadEmergenceLongDayBase.FixedValue'] =  200
    try:
        CampInputs.loc[c,'[Phenology].HeadEmergencePpSensitivity.FixedValue'] = Aus_FL_ANdata.loc[c,'[Phenology].HeadEmergencePpSensitivity.FixedValue']
    except:
        CampInputs.loc[c,'[Phenology].HeadEmergencePpSensitivity.FixedValue'] = 2
    CampInputs.loc[c,'[Phenology].CAMP.FLNparams.PpLN'] = max(0,CampInputs.loc[c,'[Phenology].CAMP.FLNparams.PpLN'])
    CampInputs.loc[c,'[Phenology].CAMP.FLNparams.VrnLN'] = max(0,CampInputs.loc[c,'[Phenology].CAMP.FLNparams.VrnLN'])

In [5]:
IttersObsPred = pd.DataFrame(columns = ['ScObs','ScPred','Var','Predicted.Simulation.Name','Predicted.Experiment','Predicted.Wheat.SowingDate',
                                                       'NSE','nSims']+paramNames,
                                                        index=pd.MultiIndex.from_arrays([[],[],[]],names=['Cultivar','itter','indloc']))
itter = 1
c='CRW247'
paramValues = [8.0,2,7,0,170,0.0]#[11.361996665701804, 7.722980823327151, 0.3034778864694383, 8.605875140401132, 107, 0.5320729781312123]#[12.749903243908449, 0.40565675723457506, 3.1181189222295616, 3.124163284147423, 58, 6.964840497000481]#[9.032310407820757, 12.0, 5.04128670051436, 2.5544490840721004, 113, 0.9682675298893731] 
# Preparefile(WheatFilePath,'Harvest')
runModelItter(paramNames,paramValues,FittingVariables,c,BaseLine,'Harvest',WheatFilePath,'test')
#Cultivars = [#'Amarok',
 # 'Axe',
 # 'Batavia',
 # 'Battenspring',
 # 'Battenwinter',
 #'Beaufort',
 # 'Bennett',
 # 'Bolac',
 # 'Braewood',
 # 'Calingiri',
 # 'Catalina',
 # 'Claire',
 # 'Condo',
 # 'Crusader',
 # 'Csirow002',
 # 'Csirow003',
 # 'Csirow005',
 # 'Csirow007',
 # 'Csirow011',
 # 'Csirow018',
 # 'Csirow021',
 # 'Csirow023',
 # 'Csirow027',
 # 'Csirow029',
 # 'Csirow073',
 # 'Csirow077',
 # 'Csirow087',
 # 'Csirow102',
 # 'Csirow105',
 # 'Cunningham',
 #'Cutlass',
 #'Derrimut',
# 'Drysdale',
#  'Eaglehawk',
#  'Egret',
#  'Ellison',
# 'Emu_rock',
#  'Forrest',
#  'Gauntlet',
#  'Gregory',
#  'Grenade',
#  'H45',
# 'Hartog',
#  'Hume',
#  'Janz',
#  'Kellalac',
#  'Kittyhawk',
#  'Lancer',
#  'Lang',
 #'Lincoln',
# 'Longsword',
#  'Mace',
#  'Mackellar',
#  'Magenta',
#  'Manning',
# 'Mccubbin',
#  'Merinda',
#  'Mitch',
#  'Otane',
#  'Ouyen',
#  'Peake',
#  'Revenue',
# 'Rongotea',
#  'Rosella',
# 'Scepter',
#  'Scout',
 # 'Scythe',
#  'Spitfire',
#  'Strzelecki',
# 'Sunbee']
#  'Sunbri',
#  'Sunco',
#  'Suneca',
#  'Sunlamb',
#  'Sunstate',
#  'Suntop',
#  'Trojan',
#  'Wakanui',
#  'Wedgetail',
#  'Whistler',
#  'Wills',
#  'Wyalkatchem',
#  'Yecora',
#  'Yitpi',
#  'Young']

Failed to find SetCropParams to delete


IndexError: index 0 is out of bounds for axis 0 with size 0

In [44]:
Version = '6'
#Kappas=pd.Series(index=[0,1,2],data=[100,10,1])
FailedCultivars = []
rounds = 0
while (rounds<1):
    for c in Cultivars:
        print ('Fitting '+c)
        try:
            checkpoint_saver = CheckpointSaver("./OptFiles/"+c+" FitsCheckpoint"+Version+".pkl", compress=9)
            Preparefile(WheatFilePath,'Harvest')
            RandomCalls = 100
            OptimizerCalls =30
            TotalCalls = RandomCalls + OptimizerCalls
            try:
                x0 = list(CampInputs.loc[c,paramNames].values)
            except:
                x0 = [8.5, 10.2, 7.0, 4.0, 120, 4.5]
            bounds = [(5.0,15.0),
                      (0.0,12.0),
                      (0.0,12.0),
                      (-10.0,10.0),
                      (50,500),
                      (0,8.0)]

            if rounds == 0:
                print ('Creating empty IttersObsPred dataframe')
                globals()['IttersObsPred'] = pd.DataFrame(columns = ['ScObs','ScPred','Var','Predicted.Simulation.Name','Predicted.Experiment','Predicted.Wheat.SowingDate',
                                                       'NSE','nSims']+paramNames,
                                                        index=pd.MultiIndex.from_arrays([[],[],[]],names=['Cultivar','itter','indloc']))
                globals()['itter'] = 0
                ret = gp_minimize(runDevModelItter, bounds, n_calls=TotalCalls,n_initial_points=RandomCalls,
                         initial_point_generator='sobol',callback=[checkpoint_saver],x0=x0)
                currentBest = ret.fun
            else:
                CheckPoint = load("./OptFiles/"+c+" FitsCheckpoint"+Version+".pkl")
                globals()['IttersObsPred'] = pd.read_pickle("./OptFiles/"+c+"_IttersObsPred"+Version+".pkl")
                globals()['itter'] = len(CheckPoint.func_vals)
                x0 = CheckPoint.x_iters
                y0 = CheckPoint.func_vals
                ret = gp_minimize(runDevModelItter, bounds, n_calls=TotalCalls,n_initial_points=RandomCalls,
                         initial_point_generator='sobol',callback=[checkpoint_saver],x0=x0,y0=y0)
                currentBest = ret.fun
            print(c)
            print(str(rounds))
            print(str(currentBest))
        except:
            FailedCultivars.append(c)
    rounds += 1

Fitting Beaufort
Failed to find DailyObsPred to delete
File Prep Complete
Creating empty IttersObsPred dataframe


C:\Anaconda\lib\site-packages\skopt\sampler\sobol.py:246: UserWarning: The balance properties of Sobol' points require n to be a power of 2. 0 points have been previously generated, then: n=0+101=101. 
  warnings.warn("The balance properties of Sobol' points require "


0  [9.0, 1.0, 4.66666666666667, 1.6666666666666607, 230.0, 0.0, -5.0] run completed 67 sims in 32 seconds.  NSE = -0.8324849433429158
1  [10.51606457286848, 7.6847540408042985, 5.362167823296543, 0.7944549815377755, 431, 3.8247023583795663, -5.0] run completed 63 sims in 31 seconds.  NSE = -8.660008525822048
2  [13.016064572868482, 4.6847540408042985, 8.362167823296543, -4.2055450184622245, 94, 1.8247023583795663, -5.0] run completed 65 sims in 32 seconds.  NSE = -7.639456183551875
3  [8.01606457286848, 10.684754040804298, 2.3621678232965433, 5.7944549815377755, 319, 5.824702358379566, -5.0] run completed 63 sims in 33 seconds.  NSE = -4.8747774193062385
4  [9.26606457286848, 6.1847540408042985, 6.862167823296543, -6.7055450184622245, 150, 6.824702358379566, -5.0] run completed 66 sims in 32 seconds.  NSE = -2.2897322271311857
5  [14.266064572868482, 0.18475404080429847, 0.8621678232965433, 3.2944549815377755, 375, 2.8247023583795663, -5.0] run completed 65 sims in 32 seconds.  NSE = -

C:\Anaconda\lib\site-packages\skopt\sampler\sobol.py:246: UserWarning: The balance properties of Sobol' points require n to be a power of 2. 0 points have been previously generated, then: n=0+101=101. 
  warnings.warn("The balance properties of Sobol' points require "


0  [5.666666666666667, 5.333333333333333, 1.0, 1.333333333333333, 230.0, 0.5, -5.0] run completed 67 sims in 30 seconds.  NSE = -0.9264858814000605
1  [14.062221754120722, 8.868994303718754, 11.646351090989123, -7.5580822055590025, 61, 6.912544061815201, -5.0] run completed 63 sims in 32 seconds.  NSE = -21.959554150050167
2  [6.562221754120721, 5.868994303718754, 2.646351090989123, 7.441917794440997, 174, 4.912544061815201, -5.0] run completed 66 sims in 32 seconds.  NSE = -5.312116425964569
3  [11.56222175412072, 11.868994303718754, 8.646351090989123, -2.5580822055590025, 399, 0.9125440618152005, -5.0] run completed 63 sims in 33 seconds.  NSE = -20.91230749287313
4  [12.81222175412072, 7.368994303718754, 1.146351090989123, 4.941917794440997, 230, 1.9125440618152005, -5.0] run completed 64 sims in 34 seconds.  NSE = -16.343592569030033
5  [7.812221754120721, 1.3689943037187557, 7.146351090989123, -5.0580822055590025, 455, 5.912544061815201, -5.0] run completed 63 sims in 33 seconds. 

...

0  [6.333333333333333, 1.666666666666667, 4.33333333333333, 0.666666666666667, 210.0, 0.0, -5.0] run completed 98 sims in 36 seconds.  NSE = -1.3261652572263678
1  [6.510011069364188, 4.202556019876592, 6.662354150308797, -1.327028296412065, 161, 6.644948156493585, -5.0] run completed 98 sims in 36 seconds.  NSE = -9.462321103959646
2  [9.010011069364188, 1.2025560198765923, 9.662354150308797, -6.327028296412065, 274, 4.644948156493585, -5.0] run completed 96 sims in 37 seconds.  NSE = -19.056748019654734
3  [14.010011069364188, 7.202556019876592, 3.6623541503087966, 3.672971703587935, 499, 0.6449481564935837, -5.0] run completed 95 sims in 37 seconds.  NSE = -28.278336780800934
4  [5.260011069364188, 2.7025560198765923, 8.162354150308797, -8.827028296412065, 330, 1.6449481564935837] gave out of bounds parameters 
5  [10.260011069364188, 8.702556019876592, 2.1623541503087966, 1.172971703587935, 105, 5.644948156493585, -5.0] run completed 96 sims in 38 seconds.  NSE = -9.992992290678801

C:\Anaconda\lib\site-packages\skopt\sampler\sobol.py:246: UserWarning: The balance properties of Sobol' points require n to be a power of 2. 0 points have been previously generated, then: n=0+101=101. 
  warnings.warn("The balance properties of Sobol' points require "


0  [6.0, 1.333333333333333, 5.333333333333337, -1.3333333333333366, 170.0, 0.0, -5.0] run completed 67 sims in 33 seconds.  NSE = 0.1504643723523873
1  [11.867333738419845, 0.2000740339928342, 8.295772385578827, -6.639958529169823, 141, 7.578485721630492, -5.0] run completed 66 sims in 32 seconds.  NSE = -13.484707156424689
2  [14.367333738419845, 9.200074033992832, 11.295772385578827, 8.360041470830176, 254, 5.578485721630492] gave out of bounds parameters 
3  [9.367333738419845, 3.200074033992834, 5.295772385578827, -1.6399585291698227, 479, 1.5784857216304928, -5.0] run completed 65 sims in 34 seconds.  NSE = -17.163108752692942
4  [10.617333738419845, 10.700074033992832, 9.795772385578827, 5.860041470830176, 310, 2.578485721630493, -5.0] run completed 63 sims in 32 seconds.  NSE = -26.09111309468702
5  [5.617333738419845, 4.700074033992834, 3.7957723855788266, -4.139958529169823, 85, 6.578485721630492] gave out of bounds parameters 
6  [13.117333738419845, 7.700074033992832, 6.7957

C:\Users\Cflhxb\AppData\Local\Temp\ipykernel_14100\4069875546.py:134: RuntimeWarning: invalid value encountered in double_scalars
  return (Value - RMin)/(RMax-RMin)
C:\Users\Cflhxb\AppData\Local\Temp\ipykernel_14100\4069875546.py:134: RuntimeWarning: invalid value encountered in double_scalars
  return (Value - RMin)/(RMax-RMin)


35  [9.836083738419845, 0.3875740339928342, 0.6082723855788252, 9.922541470830176, 345, 6.703485721630492, -5.0] run completed 64 sims in 34 seconds.  NSE = -25.999842795838347
36  [11.086083738419845, 7.887574033992832, 5.108272385578827, -2.5774585291698227, 64, 5.703485721630492, -5.0] run completed 66 sims in 33 seconds.  NSE = -8.198351373929208
37  [6.086083738419845, 1.8875740339928342, 11.108272385578827, 7.422541470830176, 289, 1.7034857216304928, -5.0] run completed 66 sims in 34 seconds.  NSE = -11.887080524134074
38  [13.586083738419845, 10.887574033992832, 2.1082723855788252, 2.4225414708301773, 176, 3.703485721630492, -5.0] run completed 64 sims in 32 seconds.  NSE = -18.635480744632737
39  [8.586083738419845, 4.887574033992834, 8.108272385578827, -7.577458529169823, 401, 7.703485721630492, -5.0] run completed 64 sims in 33 seconds.  NSE = -25.32244029575077
40  [9.211083738419845, 7.137574033992832, 11.858272385578827, 6.172541470830176, 429, 0.20348572163049283, -5.0] r

In [7]:
Cultivars = ['Whistler']

In [8]:
Version = '5'
#Kappas=pd.Series(index=[0,1,2],data=[100,10,1])
FailedCultivars = []
rounds = 0
while (rounds<3):
    for c in Cultivars:
        print ('Fitting '+c)
        try:
            checkpoint_saver = CheckpointSaver("./OptFiles/"+c+" FitsCheckpoint"+Version+".pkl", compress=9)
            Preparefile(WheatFilePath,'Harvest')
            RandomCalls = 50
            OptimizerCalls =15
            TotalCalls = RandomCalls + OptimizerCalls
            try:
                x0 = list(CampInputs.loc[c,paramNames].values)
            except:
                x0 = [8.5, 10.2, 7.0, 4.0, 120, 4.5]
            bounds = [(5.0,15.0),
                      (0.0,12.0),
                      (0.0,12.0),
                      (-10.0,10.0),
                      (50,500),
                      (0,8.0)]

            if rounds == 0:
                print ('Creating empty IttersObsPred dataframe')
                globals()['IttersObsPred'] = pd.DataFrame(columns = ['ScObs','ScPred','Var','Predicted.Simulation.Name','Predicted.Experiment','Predicted.Wheat.SowingDate',
                                                       'NSE','nSims']+paramNames,
                                                        index=pd.MultiIndex.from_arrays([[],[],[]],names=['Cultivar','itter','indloc']))
                globals()['itter'] = 0

                ret = gp_minimize(runDevModelItter, bounds, n_calls=TotalCalls,n_initial_points=RandomCalls,
                         initial_point_generator='sobol',callback=[checkpoint_saver],x0=x0)
                currentBest = ret.fun
            else:
                CheckPoint = load("./OptFiles/"+c+" FitsCheckpoint"+Version+".pkl")
                globals()['IttersObsPred'] = pd.read_pickle("./OptFiles/"+c+"_IttersObsPred"+Version+".pkl")
                globals()['itter'] = len(CheckPoint.func_vals)
                x0 = CheckPoint.x_iters
                y0 = CheckPoint.func_vals
                ret = gp_minimize(runDevModelItter, bounds, n_calls=TotalCalls,n_initial_points=RandomCalls,
                         initial_point_generator='sobol',callback=[checkpoint_saver],x0=x0,y0=y0)
                currentBest = ret.fun
            print(c)
            print(str(rounds))
            print(str(currentBest))
        except:
            FailedCultivars.append(c)
    rounds += 1

Fitting Whistler
C:\GitHubRepos\ApsimX\Tests\Validation\Wheat\Wheat.db


The process cannot access the file because it is being used by another process.


Creating empty IttersObsPred dataframe


C:\Anaconda\lib\site-packages\skopt\sampler\sobol.py:246: UserWarning: The balance properties of Sobol' points require n to be a power of 2. 0 points have been previously generated, then: n=0+51=51. 
  warnings.warn("The balance properties of Sobol' points require "


0  [8.333333333333334, 1.9999999999999964, 6.16666666666667, 0.0, 170.0, 0.0] run completed 76 sims in 30 seconds.  NSE = 0.729143669607107
1  [7.269397109146256, 0.9783781618939287, 11.735458337765714, -8.74361343205071, 51, 1.3938019526965242] run completed 76 sims in 32 seconds.  NSE = -0.4257816377714756
2  [9.769397109146256, 9.97837816189393, 2.7354583377657153, 6.256386567949292, 164, 7.393801952696525] run completed 73 sims in 32 seconds.  NSE = -2.8203323463971133
3  [14.769397109146254, 3.9783781618939287, 8.735458337765714, -3.7436134320507097, 389, 3.393801952696524] run completed 73 sims in 33 seconds.  NSE = -8.0727820771017
4  [6.019397109146256, 11.47837816189393, 1.2354583377657153, 3.7563865679492903, 220, 4.393801952696524] run completed 76 sims in 32 seconds.  NSE = -0.04838078499169485
5  [11.019397109146256, 5.478378161893929, 7.235458337765714, -6.24361343205071, 445, 0.3938019526965242] run completed 75 sims in 33 seconds.  NSE = -1.812944682503824
6  [8.5193971

C:\Anaconda\lib\site-packages\skopt\sampler\sobol.py:246: UserWarning: The balance properties of Sobol' points require n to be a power of 2. 0 points have been previously generated, then: n=0+115=115. 
  warnings.warn("The balance properties of Sobol' points require "


65  [14.271460974283883, 5.323785971450149, 7.178420170620825, -9.53249963168222, 443, 1.56878885023961] gave out of bounds parameters 
66  [6.771460974283885, 8.32378597145015, 4.178420170620825, -4.532499631682221, 106, 3.56878885023961] gave out of bounds parameters 
67  [11.771460974283883, 2.3237859714501488, 10.178420170620823, 5.467500368317777, 331, 7.56878885023961] run completed 73 sims in 35 seconds.  NSE = -8.650385444744405
68  [13.021460974283883, 6.823785971450149, 5.678420170620825, 2.967500368317779, 500, 0.5687888502396099] run completed 73 sims in 49 seconds.  NSE = -4.704035874419704
69  [8.021460974283885, 0.8237859714501488, 11.678420170620823, -7.032499631682221, 275, 4.56878885023961] run completed 75 sims in 38 seconds.  NSE = -0.7403443838503616
70  [5.521460974283885, 9.82378597145015, 8.678420170620823, -2.032499631682221, 162, 2.56878885023961] run completed 76 sims in 35 seconds.  NSE = 0.07346132863352073
71  [10.521460974283883, 3.8237859714501488, 2.678

C:\Anaconda\lib\site-packages\skopt\sampler\sobol.py:246: UserWarning: The balance properties of Sobol' points require n to be a power of 2. 0 points have been previously generated, then: n=0+180=180. 
  warnings.warn("The balance properties of Sobol' points require "


130  [6.3108855931804815, 7.412768171610408, 9.927440528694078, -6.8227636797572, 286, 3.786879295357357] run completed 76 sims in 35 seconds.  NSE = -0.32423414795763206
131  [11.310885593180481, 1.4127681716104084, 3.927440528694076, 3.177236320242802, 61, 7.786879295357357] run completed 74 sims in 35 seconds.  NSE = -0.4460082933734255
132  [12.560885593180481, 8.91276817161041, 2.427440528694076, -4.3227636797572, 230, 6.786879295357357] gave out of bounds parameters 
133  [7.5608855931804815, 2.9127681716104084, 8.427440528694076, 5.677236320242802, 455, 2.786879295357357] run completed 74 sims in 38 seconds.  NSE = -1.4942838572432917
134  [5.0608855931804815, 11.91276817161041, 5.427440528694076, -9.3227636797572, 117, 0.7868792953573571] gave out of bounds parameters 
135  [10.060885593180481, 5.912768171610408, 11.427440528694078, 0.6772363202428018, 342, 4.786879295357357] run completed 73 sims in 35 seconds.  NSE = -5.803411681265094
136  [10.685885593180481, 6.662768171610

In [ ]:
def PlotObsPre(obsPreSet,c,ret):
    graph = plt.figure(figsize=(10,20))
    gs = gridspec.GridSpec(6, 6)
    ax = graph.add_subplot(gs[0, 0:2])
    colors = pd.Series(index = ['Wheat.Phenology.FinalLeafNumber','Wheat.Phenology.FlagLeafDAS',
                        'Wheat.Phenology.HeadingDAS','Wheat.Phenology.FloweringDAS'],
                       data = ['r','b','g','k'])
    markers = ['o','s','^','v','1','2','3','4','+','x','d']

    for var in obsPreSet.loc[:,'Var'].drop_duplicates():
        epos = 0
        lablab = var.replace('Wheat.Phenology.','')
        for expt in obsPreSet.loc[:,'Predicted.Experiment'].drop_duplicates():
            filt = ((obsPreSet.loc[:,'Var'] == var) & (obsPreSet.loc[:,'Predicted.Experiment'] == expt))
            plt.plot(obsPreSet.loc[filt,'ScObs'],obsPreSet.loc[filt,'ScPred'],
                     markers[epos],color=colors[var],label=lablab)
            
            epos+=1
            if epos == 11:
                epos= 0
            lablab = None
    plt.plot([0,1],[0,1],'-',color='k')
    plt.ylabel('Scalled Predictions')
    plt.xlabel('Scalled Observations')
    plt.legend(bbox_to_anchor=(1.1, 1.05))
    RegStats = MUte.MathUtilities.CalcRegressionStats('LN',obsPreSet.loc[:,'ScPred'].values,obsPreSet.loc[:,'ScObs'].values)
    plt.text(0.97,0.03,'NSE = '+str(RegStats.NSE)[:4],transform=ax.transAxes,horizontalalignment='right')
    plt.text(0.03,0.97,c,horizontalalignment='left')
    
    ax = graph.add_subplot(gs[0, 3:6])
    plot_convergence(ret);
    plt.ylim(-1,0)
    #plt.plot([20,20,35,35,55,55,70,70,90,90,105,105,125,125,140,140,155,155,175,175,190,190,210,210,225,225]
    
    ShortParams = pd.Series(index=paramNames,data=['MinLN','PpLN','VrnLN','VxPLN','LDB','HPPS'])
    pos = 0
    for p in paramNames:
        ax = graph.add_subplot(gs[1,pos])
        plt.plot(ParamCombs.loc[:,p],-ParamCombs.loc[:,'NSE'],'o',color='k')
        bestFit = ParamCombs.loc[:,'NSE'].idxmin()
        plt.plot(ParamCombs.loc[bestFit,p],-ParamCombs.loc[bestFit,'NSE'],'o',color='cyan',ms=8,mec='k',mew=2)
        plt.plot(ret.x_iters[0][pos],-ret.func_vals[0],'o',color='orange',ms=8,mec='k',mew=2)
        if pos == 0:
            plt.ylabel('NSE')
        else:
            ax.axes.yaxis.set_visible(False)
        pos+=1
        plt.xlabel(ShortParams[p])
        plt.ylim(0,1)
    plt.tight_layout()

In [ ]:
c = 'Bennett'
IttersObsPred = pd.read_pickle("./OptFiles/"+c+"_IttersObsPred"+Version+".pkl")
ret = load("./OptFiles/"+c+" FitsCheckpoint"+Version+".pkl")
ParamCombs = pd.DataFrame(ret.x_iters,columns = paramNames)
ParamCombs.loc[:,'NSE'] = ret.func_vals
bestFitItter = ParamCombs.loc[:,'NSE'].idxmin()
bestFitObsPred = IttersObsPred.loc[(c,bestFitItter),:]
PlotObsPre(bestFitObsPred,c,ret)

In [ ]:
from skopt.plots import plot_objective
plot_objective(ret)#,minimum='expected_minimum')